In [1]:
import numpy as np 
import pandas as pd 

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
monthly_expenditure = pd.read_csv("monthly_expenditures.csv")                

# Train Test Info

In [4]:
train.head()

,musteri,yas,kidem_suresi,egitim,is_durumu,meslek_grubu,tarih,target
0,23b2476c8a,44.0,46.0,5f8ca0f83b,915be3a7a4,44af6626d6,20190601,0
1,5a7e3a7122,39.0,194.0,01a4f2c96c,915be3a7a4,44af6626d6,20190601,0
2,2ec95c7499,38.0,182.0,7e6640bfe0,d36d84f51a,44af6626d6,20190601,0
3,54399dac33,34.0,101.0,5f8ca0f83b,991c4998fb,44af6626d6,20190601,0
4,97a74b2e58,41.0,125.0,7bb291e291,242927d0f5,91ed9c1e18,20190601,0


In [ ]:
test.head()

# Monthly Info

In [ ]:
monthly_expenditure.head(10)

,musteri,sektor,islem_adedi,aylik_toplam_tutar,tarih
0,f6b1f7d2d6,DIJITAL_URUNLR,1.0,10.0,20190101.0
1,f6b1f7d2d6,EGL_SPOR_HOBI,2.0,520.0,20190101.0
2,f6b1f7d2d6,GIYIM_AKSESUAR,4.0,1420.0,20190101.0
3,f6b1f7d2d6,KUYUMCU,1.0,550.0,20190101.0
4,f6b1f7d2d6,RESTORAN_CATER,1.0,90.0,20190101.0
5,f6b1f7d2d6,TURIZM_KONAKLM,3.0,1230.0,20190101.0
6,f6b1f7d2d6,TASIMACILIK,14.0,14330.0,20190101.0
7,f6b1f7d2d6,BENZIN_YAKIT,1.0,150.0,20190101.0
8,f6b1f7d2d6,DIJITAL_URUNLR,1.0,10.0,20190201.0
9,f6b1f7d2d6,BENZIN_YAKIT,1.0,150.0,20190201.0


* İlk aklıma gelen şeyler kendime notlar
* Train içinde olup monthly tablosunda olmayan var mı?
* Test içinde olup monthly tablosunda olmayan var mı
*  Monthly, bir müşterinin her ay için 7 farklı kategoride işlem adedini ve aylık toplam tutarını belirtiyor.

In [ ]:
monthly_expenditure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197961 entries, 0 to 197960
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   musteri             197961 non-null  object 
 1   sektor              197960 non-null  object 
 2   islem_adedi         197960 non-null  float64
 3   aylik_toplam_tutar  197960 non-null  float64
 4   tarih               197960 non-null  float64
dtypes: float64(3), object(2)
memory usage: 7.6+ MB


In [ ]:
monthly_expenditure["sektor"].value_counts()

RESTORAN_CATER    53922
GIYIM_AKSESUAR    36536
BENZIN_YAKIT      26362
ELKT_ESYA_BILG    18027
TASIMACILIK       14196
TURIZM_KONAKLM    11447
EGL_SPOR_HOBI     10723
DIJITAL_URUNLR     8686
SAGLIK_HIZMTLR     5836
ICKILI_YERLER      4252
BAHCE_CICEKCLK     3533
KLP_DERNK_SOSY     2365
KUYUMCU            2075
Name: sektor, dtype: int64

In [3]:
six_month = monthly_expenditure.groupby(["musteri"]).mean()
six_month.columns = ["6_ort_islem",  "6_ortalama_tutar","delete"]
six_month = six_month.drop("delete",axis=1)
six_month.head(5)

,6_ort_islem,6_ortalama_tutar
musteri,,
0001791170,2.214286,273.571429
00088d74bd,4.344828,5322.068966
000d648614,1.555556,133.333333
0014450f7c,3.285714,930.000000
0015501e39,4.050000,607.500000


In [4]:
three_month = monthly_expenditure[monthly_expenditure["tarih"] > 20190302]
three_month = three_month.groupby(["musteri"]).mean()
three_month.columns = ["3_ort_islem",  "3_ortalama_tutar","delete"]
three_month = three_month.drop("delete",axis=1)
three_month.head(5)                                                     # barış:aşağıdaki tablonun ilk satırındaki müşteri (0001791170) month_exp.csv de yok. nedenini bilmiyorum.

,3_ort_islem,3_ortalama_tutar
musteri,,
0001791170,1.5,90.000000
00088d74bd,4.2,4882.000000
000d648614,3.0,370.000000
0014450f7c,4.5,1553.333333
0015501e39,3.6,577.000000


In [5]:
six_month_sektor = monthly_expenditure.groupby(["musteri","sektor"]).mean()
six_month_sektor.columns = ["_6",  "_6","delete"]
six_month_sektor = six_month_sektor.drop("delete",axis=1) # barış:önemi yok ama şu delete-drop olayını anlamadım bi ara söylersen sevinirim:)
six_month_sektor.head(50)

_6            _6
musteri    sektor                                 
0001791170 BENZIN_YAKIT     3.000000    420.000000
           GIYIM_AKSESUAR   1.250000     42.500000
           ICKILI_YERLER    1.000000    180.000000
           RESTORAN_CATER   2.600000    360.000000
00088d74bd BAHCE_CICEKCLK   1.000000    140.000000
           BENZIN_YAKIT     2.000000    980.000000
           ELKT_ESYA_BILG   2.500000   1910.000000
           GIYIM_AKSESUAR   4.500000  10026.666667
           KLP_DERNK_SOSY   1.000000    350.000000
           RESTORAN_CATER   9.833333   7616.666667
           SAGLIK_HIZMTLR   3.500000   6775.000000
           TASIMACILIK      1.000000     50.000000
           TURIZM_KONAKLM   1.800000    498.000000
000d648614 BENZIN_YAKIT     1.000000     60.000000
           DIJITAL_URUNLR   1.000000    120.000000
           ELKT_ESYA_BILG   1.000000     45.000000
           RESTORAN_CATER   2.250000    212.500000
           TURIZM_KONAKLM   1.000000     80.000000
0014450f7c GIYIM_AKSESUAR   1.600000    266.000000
           RESTORAN_CATER   5.500000   1806.666667
           SAGLIK_HIZMTLR   3.000000    810.000000
           TASIMACILIK      1.000000     20.000000
           TURIZM_KONAKLM   1.000000     20.000000
0015501e39 BAHCE_CICEKCLK   1.000000     50.000000
           BENZIN_YAKIT     1.000000     50.000000
           EGL_SPOR_HOBI    1.500000     25.000000
           GIYIM_AKSESUAR   1.000000     20.000000
           ICKILI_YERLER    1.000000     30.000000
           KLP_DERNK_SOSY   1.000000     10.000000
           RESTORAN_CATER  10.333333   1850.000000
           TASIMACILIK      1.800000    154.000000
           TURIZM_KONAKLM   1.000000     40.000000
0015f58946 BAHCE_CICEKCLK   1.000000     70.000000
           BENZIN_YAKIT     2.800000    326.000000
           ELKT_ESYA_BILG   2.000000    340.000000
           GIYIM_AKSESUAR   1.500000    435.000000
           RESTORAN_CATER   2.400000    508.000000
           TASIMACILIK      1.000000     15.000000
           TURIZM_KONAKLM   1.000000    286.666667
001c64c583 BENZIN_YAKIT     2.666667    640.000000
           GIYIM_AKSESUAR   2.000000    390.000000
           RESTORAN_CATER   1.000000     70.000000
0023c5e289 BENZIN_YAKIT     5.000000   1605.000000
           ELKT_ESYA_BILG   1.333333   2120.000000
           GIYIM_AKSESUAR   1.000000    125.000000
           ICKILI_YERLER    1.000000    120.000000
           RESTORAN_CATER   1.750000    185.000000
           TURIZM_KONAKLM   1.000000    140.000000
002e74bd21 EGL_SPOR_HOBI    1.000000    170.000000
           ELKT_ESYA_BILG   3.000000    860.000000

In [6]:
ana_kategoriler = list(monthly_expenditure["sektor"].unique())
altı_kategoriler = list(monthly_expenditure["sektor"].unique())
for i in range (len(ana_kategoriler)):
    altı_kategoriler[i] = "6_"+altı_kategoriler[i]

In [7]:
df_list = []

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[0]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[0]+"_islem_6",  ana_kategoriler[0]+"_tutar_6","delete"]
dijital = temp.drop("delete",axis=1)
df_list.append(dijital)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[1]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[1]+"_islem_6",  ana_kategoriler[1]+"_tutar_6","delete"]
spor = temp.drop("delete",axis=1)
df_list.append(spor)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[2]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[2]+"_islem_6",  ana_kategoriler[2]+"_tutar_6","delete"]
giyim = temp.drop("delete",axis=1)
df_list.append(giyim)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[3]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[3]+"_islem_6",  ana_kategoriler[3]+"_tutar_6","delete"]
kuyumcu = temp.drop("delete",axis=1)
df_list.append(kuyumcu)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[4]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[4]+"_islem_6",  ana_kategoriler[4]+"_tutar_6","delete"]
restoran = temp.drop("delete",axis=1)
df_list.append(restoran)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[5]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[5]+"_islem_6",  ana_kategoriler[5]+"_tutar_6","delete"]
turizm = temp.drop("delete",axis=1)
df_list.append(turizm)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[6]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[6]+"_islem_6",  ana_kategoriler[6]+"_tutar_6","delete"]
tasimacilik = temp.drop("delete",axis=1)
df_list.append(tasimacilik)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[7]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[7]+"_islem_6",  ana_kategoriler[7]+"_tutar_6","delete"]
yakıt = temp.drop("delete",axis=1)
df_list.append(yakıt)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[8]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[8]+"_islem_6",  ana_kategoriler[8]+"_tutar_6","delete"]
elek_esya = temp.drop("delete",axis=1)
df_list.append(elek_esya)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[9]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[9]+"_islem_6",  ana_kategoriler[9]+"_tutar_6","delete"]
saglik = temp.drop("delete",axis=1)
df_list.append(saglik)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[10]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[10]+"_islem_6",  ana_kategoriler[10]+"_tutar_6","delete"]
dernek = temp.drop("delete",axis=1)
df_list.append(dernek)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[11]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[11]+"_islem_6",  ana_kategoriler[11]+"_tutar_6","delete"]
alkol = temp.drop("delete",axis=1)
df_list.append(alkol)

temp = monthly_expenditure[monthly_expenditure["sektor"] == ana_kategoriler[12]]
temp = temp.groupby(["musteri"]).mean()
temp.columns = [ana_kategoriler[12]+"_islem_6",  ana_kategoriler[12]+"_tutar_6","delete"]
bahce = temp.drop("delete",axis=1)
df_list.append(bahce)

In [ ]:
df_list[12]

,BAHCE_CICEKCLK_islem_6,BAHCE_CICEKCLK_tutar_6
musteri,,
00088d74bd,1.0,140.0
0015501e39,1.0,50.0
0015f58946,1.0,70.0
00419a049b,1.0,80.0
004c505af3,1.5,155.0
...,...,...
ffa67aaeb4,1.0,70.0
ffa8b62d39,1.0,70.0
ffbb3a00d9,1.0,60.0


In [8]:

# print(max(train["kidem_suresi"])) # barış: kıdem süresi büyük ihtimalle ay cinsinden. her neyse önemi yok zaten herkes için aynı.
# Berke: Abi kidem_süresine baktığımzda -1 olanları görüyoruz onları average yapalım
train.loc[train['kidem_suresi'] == -1, 'kidem_suresi'] = train["kidem_suresi"].mean()
test.loc[train['kidem_suresi'] == -1, 'kidem_suresi'] = test["kidem_suresi"].mean()

In [9]:
train["meslek_grubu"].value_counts()
test["meslek_grubu"].value_counts()

44af6626d6    18135
314826bda8     1637
91ed9c1e18     1033
ba91c4a92b     1001
bfd9cb6270      704
51155d7cc3      556
b18c21407b      433
6da8606d6e      410
62260c15fb      287
4e1d23cb0a      257
527516f307      252
419188ba43      242
7330d6bdb3      192
9034d4c83c      192
8bbaa7ab87      102
51017102ac       85
070e3be3ae       65
8a8492b947       36
eea19ef68d       34
5aaf0d2c89        9
Name: meslek_grubu, dtype: int64

In [10]:
train.loc[train['meslek_grubu'] == "44af6626d6", 'meslek'] = 1
train.loc[train['meslek_grubu'] != "44af6626d6", 'meslek'] = 0



test.loc[test['meslek_grubu'] == "44af6626d6", 'meslek'] = 1
test.loc[test['meslek_grubu'] != "44af6626d6", 'meslek'] = 0



In [ ]:
train["is_durumu"].value_counts()

991c4998fb    24159
d36d84f51a    15820
915be3a7a4     6090
ba7b390fc4     5145
242927d0f5     2792
51be29729b     2645
eb35a5eb6b     1570
289777e76d      810
b026b8ee68      514
f1fcd26d00      197
a996720382      159
83a26fc2fd       27
3773727d6e        1
Name: is_durumu, dtype: int64

In [11]:
train.loc[train['is_durumu'] != "991c4998fb", 'is'] = "0"
train.loc[train['is_durumu'] == "991c4998fb", 'is'] = "1"




test.loc[test['is_durumu'] != "991c4998fb", 'is'] = "0"
test.loc[test['is_durumu'] == "991c4998fb", 'is'] = "1"




In [ ]:
train.head()

In [12]:
train.drop("tarih",axis=1,inplace=True)
test.drop("tarih",axis=1,inplace=True)

In [12]:
test.isna().sum()

musteri             0
yas                 0
kidem_suresi        0
egitim            208
is_durumu          54
meslek_grubu    14338
meslek              0
is                  0
dtype: int64

In [13]:
train.drop("meslek_grubu",axis=1,inplace=True)
test.drop("meslek_grubu",axis=1,inplace=True)


#train.drop("is_durumu",axis=1,inplace=True)
#test.drop("is_durumu",axis=1,inplace=True)

In [14]:
train = train.dropna()
test = test.dropna()

In [ ]:
train.isna().sum()

# Yapılacaklar
* Train ve test datalarındaki stringleri encode et
* Çok yüksek harcaması olan reisi at gitsin
* Birleşmeden önce bir visualization yap belki 
* İş durumunda(+kıdem süresinde belki) clustering 

*> Encoding categorical variables*

In [ ]:
train.head(5)
#train.info()

In [15]:
train = train.set_index('musteri')
test = test.set_index('musteri')


In [ ]:
train.head(5)

# Train to new_train

In [16]:
new_train = train
new_test = test

for i in df_list:
    new_train = new_train.merge(i, on='musteri', how='left')
    new_test = new_test.merge(i, on='musteri', how='left')
    
new_train = new_train.merge(six_month,on="musteri",how="left")
new_test = new_test.merge(six_month,on="musteri",how="left")

new_train = new_train.fillna(0)  
new_test = new_test.fillna(0)

In [17]:
y = new_train["target"]
X = new_train.drop("target",axis = 1)
X = pd.get_dummies(X)
new_test = pd.get_dummies(new_test)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train , y_test = train_test_split(X ,y,test_size = 0.25, random_state=41)


In [ ]:
X_train.head()

In [ ]:
new_test.head()

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
"""# example of grid searching key hyperparameters for gradient boosting on a classification dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)
# define the model with default hyperparameters
model = GradientBoostingClassifier()
# define the grid of values to search
grid = dict()
grid['n_estimators'] = [10, 50, 100]
grid['learning_rate'] = [0.03, 0.001, 0.01, 0.1]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 7, 9]
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',verbose=2)
# execute the grid search
grid_result = grid_search.fit(X_train2, y_train)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))"""

In [ ]:
#b = data_classification(X, y)

In [ ]:
#b.split(42)

In [ ]:
#b.default_processes()

In [ ]:
#from sklearn.tree import DecisionTreeClassifier

In [ ]:
#b.grid_all(["lgbm"])

In [ ]:
#regression = RandomForestClassifier(criterion = 'gini', n_estimators=1000,n_jobs =-1)
#regression.fit(X_train, y_train)

In [ ]:
#y_probs = regression.predict_proba(X_test)
#y_probs = y_probs[:,1]
#a = 0.01
#temp1 = []
#temp2 = []
#while a <0.7:
    #y_pred = [1 if i > a else 0 for i in y_probs]
    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    #temp1.append(metrics.auc(fpr, tpr))
    #temp2.append(a)
    #a += 0.01
    
#zipped_lists = zip(temp1, temp2)
#sorted_pairs = sorted(zipped_lists, reverse=True)


In [ ]:
#sorted_pairs[:3]

In [ ]:
#xgb_model = XGBClassifier().fit(X_train,y_train)
#lgbm_model = LGBMClassifier().fit(X_train,y_train)
#cat_model = CatBoostClassifier().fit(X_train,y_train)

In [ ]:
#roc_auc_score(y_test, xgb_model.predict_proba(X_test2)[:, 1])

In [ ]:
#roc_auc_score(y_test, lgbm_model.predict_proba(X_test2)[:, 1])

In [ ]:
#roc_auc_score(y_test, cat_model.predict_proba(X_test2)[:, 1])

In [ ]:
#y_probs = lgbm_model.predict_proba(X_test)
#y_probs = y_probs[:,1]
#a = 0.01
#temp1 = []
#temp2 = []
#while a <0.7:
    #y_pred = [1 if i > a else 0 for i in y_probs]
    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    #temp1.append(metrics.auc(fpr, tpr))
    #temp2.append(a)
   # a += 0.01
    
#zipped_lists = zip(temp1, temp2)
#sorted_pairs = sorted(zipped_lists, reverse=True)
#sorted_pairs[:3]

In [ ]:
#y_probs = cat_model.predict_proba(X_test)
#y_probs = y_probs[:,1]
#a = 0.01
#temp1 = []
#temp2 = []
#while a <0.7:
    #y_pred = [1 if i > a else 0 for i in y_probs]
    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    #temp1.append(metrics.auc(fpr, tpr))
    #temp2.append(a)
    #a += 0.01
    
#zipped_lists = zip(temp1, temp2)
#sorted_pairs = sorted(zipped_lists, reverse=True)
#sorted_pairs[:3]

In [ ]:
#y_probs = xgb_model.predict_proba(X_test)
#y_probs = y_probs[:,1]
#a = 0.01
#temp1 = []
#temp2 = []
#while a <0.7:
    #y_pred = [1 if i > a else 0 for i in y_probs]
    #fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
    #temp1.append(metrics.auc(fpr, tpr))
    #temp2.append(a)
    #a += 0.01
    
#zipped_lists = zip(temp1, temp2)
#sorted_pairs = sorted(zipped_lists, reverse=True)
#sorted_pairs[:3]

# Output


In [ ]:
#lgbm_model = LGBMClassifier().fit(X, y)

In [ ]:
#y_probs = lgbm_model.predict_proba(new_test)
#y_probs = y_probs[:,1]
#y_pred = [1 if i > 0.04 else 0 for i in y_probs]

In [ ]:
#test1 = pd.read_csv("test.csv")
#lists = [list(test1["musteri"]),y_pred]
#df =  pd.DataFrame({"musteri":list(test1["musteri"]),"target":y_pred})


In [ ]:
#df.head()

In [ ]:
#df.to_csv('results.csv', header = True,index=False)

In [ ]:
X_train.corr()

kernel PCA (dimension reduction and feature extraction)

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [24]:
classifier_rf = RandomForestClassifier(n_estimators = 1000, criterion = 'entropy', random_state = 41,bootstrap= True,
 max_depth= 80,
 max_features= 1,
 min_samples_leaf= 3,
 min_samples_split= 10)
classifier_rf.fit(X_train_pca, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=80, max_features=1,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=41, verbose=0,
                       warm_start=False)

In [26]:
y_probs_rfpca = classifier_rf.predict_proba(X_test_pca)

y_probs_rfpca = y_probs_rfpca[:,1]
a = 0.01
temp5 = []
temp6 =[]
while a <0.7:
    y_pred_rfpca = [1 if i > a else 0 for i in y_probs_rfpca]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_rfpca, pos_label=1)
    temp5.append(metrics.auc(fpr, tpr))
    temp6.append(a)
    a += 0.01
    
zipped_lists_rf5 = zip(temp5, temp6)
sorted_pairs_rf5 = sorted(zipped_lists_rf5, reverse=True)

[(0.6194521396399078, 0.01),
 (0.6193247900397854, 0.02),
 (0.6049183367713759, 0.03),
 (0.5989818220942749, 0.04),
 (0.5832878286631339, 0.05),
 (0.5762775307221983, 0.060000000000000005),
 (0.5645105466892198, 0.07),
 (0.549605049613976, 0.08),
 (0.5455267679337992, 0.09),
 (0.5427342411415856, 0.09999999999999999),
 (0.5425711860461019, 0.10999999999999999),
 (0.5424058696025787, 0.11999999999999998),
 (0.5413687439806485, 0.12999999999999998),
 (0.538485644248535, 0.13999999999999999),
 (0.536227985776835, 0.15),
 (0.5311664937727235, 0.16),
 (0.5276697355936657, 0.17),
 (0.5235824085213308, 0.18000000000000002),
 (0.5215665952715449, 0.19000000000000003),
 (0.5195670875313078, 0.20000000000000004),
 (0.5141546105077941, 0.21000000000000005),
 (0.5106392854711483, 0.22000000000000006),
 (0.5085165937719143, 0.23000000000000007),
 (0.5073191504758113, 0.25000000000000006),
 (0.5071819223553059, 0.26000000000000006),
 (0.5062381070945866, 0.24000000000000007),
 (0.5047988185765695, 0

random forest trial

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train3 = sc.fit_transform(X_train)
X_test3 = sc.transform(X_test)

In [ ]:
classifier_rf = RandomForestClassifier(n_estimators = 5000, criterion = 'entropy', random_state = 41,bootstrap= True,
 max_depth= 80,
 max_features= 1,
 min_samples_leaf= 3,
 min_samples_split= 10)
classifier_rf.fit(X_train3, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=80, max_features=1,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=5000,
                       n_jobs=None, oob_score=False, random_state=41, verbose=0,
                       warm_start=False)

In [ ]:
y_pred_rf = classifier_rf.predict(X_test3)

In [ ]:
y_probs_rf = classifier_rf.predict_proba(X_test3)

y_probs_rf = y_probs_rf[:,1]
a = 0.01
temp3 = []
temp4 =[]
while a <0.7:
    y_pred_rf = [1 if i > a else 0 for i in y_probs_rf]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_rf, pos_label=1)
    temp3.append(metrics.auc(fpr, tpr))
    temp4.append(a)
    a += 0.01
    
zipped_lists_rf3 = zip(temp3, temp4)
sorted_pairs_rf3 = sorted(zipped_lists_rf3, reverse=True)

[(0.6704092577627572, 0.04),
 (0.6655985676254498, 0.05),
 (0.6575458473907954, 0.03),
 (0.6248400105668572, 0.02),
 (0.6081467236731073, 0.060000000000000005),
 (0.5634560170811364, 0.07),
 (0.5429286295221486, 0.08),
 (0.5253778008914242, 0.09),
 (0.5207642645262607, 0.01),
 (0.5160062137150139, 0.09999999999999999),
 (0.5104185248836167, 0.10999999999999999),
 (0.5063441760596994, 0.11999999999999998),
 (0.5031139248238938, 0.12999999999999998),
 (0.5023570720447161, 0.13999999999999999),
 (0.5018623192432154, 0.15),
 (0.5006983240223464, 0.19000000000000003),
 (0.5006983240223464, 0.18000000000000002),
 (0.5006401608163905, 0.17),
 (0.5005238344044786, 0.16),
 (0.5, 0.6900000000000004),
 (0.5, 0.6800000000000004),
 (0.5, 0.6700000000000004),
 (0.5, 0.6600000000000004),
 (0.5, 0.6500000000000004),
 (0.5, 0.6400000000000003),
 (0.5, 0.6300000000000003),
 (0.5, 0.6200000000000003),
 (0.5, 0.6100000000000003),
 (0.5, 0.6000000000000003),
 (0.5, 0.5900000000000003),
 (0.5, 0.58000000000

feature importance

In [ ]:
import matplotlib.pyplot as plt

importance = []
if classifier_rf is not None:
    importance = classifier_rf.feature_importances_
    importance = pd.DataFrame(importance, index=X_train3[1], 
                              columns=["Importance"])

    importance["Std"] = np.std([classifier_rf.feature_importances_
                                for tree in classifier_rf.estimators_], axis=0)

    x = range(importance.shape[0])
    y = importance.iloc[:, 0] 
    yerr = importance.iloc[:, 1] 

    plt.bar(x, y, yerr=yerr, align="center")
    plt.xlabel('features')
    plt.ylabel('Feature importance')
    plt.title('Importance of different features')
    plt.show()

trying to improve random forest

In [ ]:
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
#Use the random grid to search for best hyperparameters UZUN SÜRÜYOR

rf_random = RandomizedSearchCV(estimator = classifier_rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=41, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train3,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 16.8min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 92.9min


2. grid search aşağıda

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [1,2,3],
    'min_samples_leaf': [3, 5],
    'min_samples_split': [10, 12],
    'n_estimators': [300, 1000]
}
# Create a based model

grid_search = GridSearchCV(estimator = classifier_rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train3, y_train)
grid_search.best_params_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 22.7min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 54.6min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 73.0min finished


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 1,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 300}

Neural Network trial

In [28]:
import tensorflow as tf



# Part 2 - Building the ANN

# Initializing the ANN
ann = tf.keras.models.Sequential()

# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding the output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the ANN

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the ANN on the Training set
ann.fit(X_train_pca, y_train, batch_size = 32, epochs = 100)


Epoch 1/100
1400/1400 [==============================] - 3s 1ms/step - loss: 0.3949 - accuracy: 0.8836
Epoch 2/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1764 - accuracy: 0.9583
Epoch 3/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1699 - accuracy: 0.9588
Epoch 4/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1731 - accuracy: 0.9572
Epoch 5/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1704 - accuracy: 0.9584
Epoch 6/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1704 - accuracy: 0.9582
Epoch 7/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1685 - accuracy: 0.9585
Epoch 8/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1684 - accuracy: 0.9585
Epoch 9/100
1400/1400 [==============================] - 2s 1ms/step - loss: 0.1686 - accuracy: 0.9584
Epoch 10/100
1400/1400 [==============================] - 2s 1ms/step - l

In [30]:
from sklearn import metrics
y_probs_ann = ann.predict_proba(X_test_pca)
y_probs_ann = y_probs_ann[:,]
y_pred_ann = [1 if i > 0.032 else 0 for i in y_probs_ann]
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_ann, pos_label=1)
metrics.auc(fpr, tpr)



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


0.6447419849494196

In [ ]:
test1= pd.read_csv("test.csv")

lists = [list(test1["musteri"]),y_pred_ann]
df =  pd.DataFrame({"musteri":list(test1["musteri"]),"target":y_pred_ann})
df.to_csv('results_ann.csv', header = True,index=False)